# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [5]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

result = 0
for i, sqrt_value in enumerate(generator):
    print(sqrt_value)
    result += sqrt_value
print(result) 
# Q1 8.382332347441762



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
32.854555867161245


In [7]:
limit = 13
generator = square_root_generator(limit)

result = 0
for i, sqrt_value in enumerate(generator, start = 1):
    if i == 13:
        print(sqrt_value)

# Q2 3.605551275463989
    


3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [22]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [9]:
#Install the dependencies
# %%capture
!pip install dlt[duckdb]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 548.6 kB/s eta 0:00:00 kB/s eta 0:00:0101
  Using cached wheel-0.42.0-py3-none-any.whl.metadata (2.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.5 MB/s eta 0:00:002.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 3.2 MB/s eta 0:00:001 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 3.4 MB/s eta 0:00:0014.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 5.4 MB/s eta 0:00:000:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 1.4 MB/s eta 0:00:00
Using cached wheel-0.42.0-py3-none-any.whl (65 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="person1",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)



Pipeline dlt_ipykernel_launcher load step completed in 0.51 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////workspaces/ZoomCamp2024/workshop1_data_ingestion/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707836090.481978 is LOADED and contains no failed jobs


In [20]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n person1 table below:")

person1 = conn.sql("SELECT * FROM person1").df()
display(person1)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ person1             │
└─────────────────────┘




 person1 table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707836090.481978,/ScZlxSRG8eqTg,None
1,2,Person_2,27,City_A,1707836090.481978,uojOoo3PerwlwA,None
2,3,Person_3,28,City_A,1707836090.481978,mHcx2rNw7gGffQ,None
3,4,Person_4,29,City_A,1707836090.481978,xbPkajGZ4LDHYg,None
4,5,Person_5,30,City_A,1707836090.481978,J2ZBgDycnpWAvQ,None
5,3,Person_3,33,City_B,1707836537.8145154,7VEtpIzFOC7C3Q,Job_3
6,4,Person_4,34,City_B,1707836537.8145154,SQYI2h+1Cd6aPQ,Job_4
7,5,Person_5,35,City_B,1707836537.8145154,JmVIPnqgfyOcMQ,Job_5
8,6,Person_6,36,City_B,1707836537.8145154,hL4Wlw1/ViJibw,Job_6
9,7,Person_7,37,City_B,1707836537.8145154,qYEZW+zQQQOenw,Job_7


In [15]:
duckdb.sql(f"select sum(age) from person1")

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

In [16]:
# Append data
info = generators_pipeline.run(people_2(),
										table_name="person1",
										write_disposition="append")
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////workspaces/ZoomCamp2024/workshop1_data_ingestion/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707836537.8145154 is LOADED and contains no failed jobs


In [ ]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n person1 table below:")

person1 = conn.sql("SELECT * FROM person1").df()
display(person1)

In [21]:
duckdb.sql(f"select sum(age) from person1")
# Q3 353

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

In [23]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="person1",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)



Pipeline dlt_ipykernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////workspaces/ZoomCamp2024/workshop1_data_ingestion/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707837877.6694658 is LOADED and contains no failed jobs


In [24]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n person1 table below:")

person1 = conn.sql("SELECT * FROM person1").df()
display(person1)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ person1             │
└─────────────────────┘




 person1 table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707837877.6694658,kRt8fm8TvQcsNA,None
1,2,Person_2,27,City_A,1707837877.6694658,XQrGtzSpB53q4g,None
2,3,Person_3,28,City_A,1707837877.6694658,WVsHP7XjcjMJXw,None
3,4,Person_4,29,City_A,1707837877.6694658,ArJeVHHt1GiPog,None
4,5,Person_5,30,City_A,1707837877.6694658,RJYOuzCKfDcuaQ,None


In [26]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')



# run with merge write disposition.
# This is so scaffolding is created for the next example,
# where we look at merging data

info = pipeline.run(people_2(),
										table_name="person1",
										write_disposition="merge",
                    primary_key="id")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.46 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////workspaces/ZoomCamp2024/workshop1_data_ingestion/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707838062.79119 is LOADED and contains no failed jobs


In [27]:
# show the outcome

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n Rides table below: Note the times are properly typed")
person1 = conn.sql("SELECT * FROM person1").df()
display(person1)



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ person1             │
└─────────────────────┘




 Rides table below: Note the times are properly typed


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707837877.6694658,kRt8fm8TvQcsNA,None
1,2,Person_2,27,City_A,1707837877.6694658,XQrGtzSpB53q4g,None
2,8,Person_8,38,City_B,1707838062.79119,7XGydAdYsihOcQ,Job_8
3,4,Person_4,34,City_B,1707838062.79119,BXnqPzOEJhR9Dw,Job_4
4,5,Person_5,35,City_B,1707838062.79119,wBjUHy89INRhpg,Job_5
5,7,Person_7,37,City_B,1707838062.79119,CqYP6wlVzVlWwA,Job_7
6,3,Person_3,33,City_B,1707838062.79119,DQp4v/Xy2eHKPg,Job_3
7,6,Person_6,36,City_B,1707838062.79119,zcYCROibnvh4Ng,Job_6


In [28]:
duckdb.sql(f" select count(1) from person1")

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│        8 │
└──────────┘

In [29]:
duckdb.sql(f" select age from person1 where id = 3")
# 33

┌───────┐
│  age  │
│ int64 │
├───────┤
│    33 │
└───────┘

In [30]:
duckdb.sql(f" select sum(age) from person1")
# 266

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX